The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [1]:
!pip install d2l==0.14.3


     |████████████████████████████████| 61kB 3.8MB/s 


# Concise Implementation of Linear Regression
:label:`sec_linear_concise`

Broad and intense interest in deep learning for the past several years
has inspired companies, academics, and hobbyists
to develop a variety of mature open source frameworks
for automating the repetitive work of implementing
gradient-based learning algorithms.
In :numref:`sec_linear_scratch`, we relied only on
(i) tensors for data storage and linear algebra;
and (ii) auto differentiation for calculating gradients.
In practice, because data iterators, loss functions, optimizers,
and neural network layers
are so common, modern libraries implement these components for us as well.

In this section, we will show you how to implement
the linear regression model from :numref:`sec_linear_scratch`
concisely by using high-level APIs of deep learning frameworks.


## Generating the Dataset

To start, we will generate the same dataset as in :numref:`sec_linear_scratch`.


In [2]:
from d2l import tensorflow as d2l
import numpy as np
import tensorflow as tf

In [3]:
true_w = tf.constant([2, -3.4])
true_b = 4.2
#d2l's own implementation obtaining synthetic data
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## Reading the Dataset

Rather than rolling our own iterator,
we can call upon the existing API in a framework to read data.
We pass in `features` and `labels` as arguments and specify `batch_size`
when instantiating a data iterator object.
Besides, the boolean value `is_train`
indicates whether or not
we want the data iterator object to shuffle the data
on each epoch (pass through the dataset).


[https://www.tensorflow.org/guide/data](https://www.tensorflow.org/guide/data)

In [4]:
dset = tf.data.Dataset.from_tensor_slices((features, labels))

In [6]:
fel = list(dset.as_numpy_iterator())

In [8]:
print(fel[0])

(array([-0.89466023, -0.6338276 ], dtype=float32), array([4.566094], dtype=float32))


In [9]:
fset = tf.data.Dataset.from_tensor_slices(features)
flist = list(fset.as_numpy_iterator())
print(flist[0])

[-0.89466023 -0.6338276 ]


In [10]:
lset = tf.data.Dataset.from_tensor_slices(labels)
llist = list(lset.as_numpy_iterator())

In [11]:
print(llist[0])

[4.566094]


In [14]:
print(type(features))
print(type(labels))

<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>


Every time `load_array` is called, if `is_train` is set to `True`, the dataset gets shuffled before returning the dataset.

In [15]:
#data_arrays is a tuple of features and labels
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """Construct a TensorFlow data iterator."""
    #basically takes 10 (batch size) rows of features and labels
    #and represents them as a "dataset"
    dataset = tf.data.Dataset.from_tensor_slices(data_arrays)
    if is_train:
        #shuffles the data by sampling entries (buffer_size)
        #from a 1000 entry buffer. Everytime an entry is sampled, the
        #entry's place is taken by a new entry from outside the buffer if any
        dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size)
    return dataset

In [16]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [23]:
#practice
dlist = list(data_iter.as_numpy_iterator())
dlist[0]

(array([[ 0.70862526,  1.2192032 ],
        [ 0.7843208 , -0.2750216 ],
        [-0.6846534 ,  0.50200665],
        [ 1.6817191 ,  1.2582612 ],
        [ 1.6341274 ,  0.30543736],
        [ 0.3761744 , -0.01750514],
        [-1.3755399 , -0.52630264],
        [-0.2920044 , -0.07757249],
        [ 0.01635778, -1.5822426 ],
        [-0.46637008,  0.9045051 ]], dtype=float32), array([[1.4664328 ],
        [6.715562  ],
        [1.1387159 ],
        [3.2866995 ],
        [6.4292264 ],
        [5.001237  ],
        [3.2359211 ],
        [3.8777082 ],
        [9.597429  ],
        [0.19003361]], dtype=float32))

In [21]:
type(data_iter)

tensorflow.python.data.ops.dataset_ops.BatchDataset

Now we can use `data_iter` in much the same way as we called
the `data_iter` function in :numref:`sec_linear_scratch`.
To verify that it is working, we can read and print
the first minibatch of examples.
Comparing with :numref:`sec_linear_scratch`,
here we use `iter` to construct a Python iterator and use `next` to obtain the first item from the iterator.


In [17]:
next(iter(data_iter))

(<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
 array([[-0.19441114,  0.49152496],
        [-0.702771  ,  1.2664042 ],
        [-1.1907264 , -0.9990662 ],
        [ 0.07564203,  0.8183015 ],
        [ 0.46178213,  0.67119825],
        [ 0.14035554, -0.87641287],
        [ 1.6341274 ,  0.30543736],
        [-0.00917123, -0.5065995 ],
        [ 1.3714517 , -0.09719807],
        [-1.3521336 , -0.3148818 ]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[ 2.1520443],
        [-1.510767 ],
        [ 5.2170906],
        [ 1.5546464],
        [ 2.8351681],
        [ 7.4624496],
        [ 6.4292264],
        [ 5.8878107],
        [ 7.258221 ],
        [ 2.566607 ]], dtype=float32)>)

## Defining the Model

When we implemented linear regression from scratch
in :numref:`sec_linear_scratch`,
we defined our model parameters explicitly
and coded up the calculations to produce output
using basic linear algebra operations.
You *should* know how to do this.
But once your models get more complex,
and once you have to do this nearly every day,
you will be glad for the assistance.
The situation is similar to coding up your own blog from scratch.
Doing it once or twice is rewarding and instructive,
but you would be a lousy web developer
if every time you needed a blog you spent a month
reinventing the wheel.

For standard operations, we can use a framework's predefined layers,
which allow us to focus especially
on the layers used to construct the model
rather than having to focus on the implementation.
We will first define a model variable `net`,
which will refer to an instance of the `Sequential` class.
The `Sequential` class defines a container
for several layers that will be chained together.
Given input data, a `Sequential` instance passes it through
the first layer, in turn passing the output
as the second layer's input and so forth.
In the following example, our model consists of only one layer,
so we do not really need `Sequential`.
But since nearly all of our future models
will involve multiple layers,
we will use it anyway just to familiarize you
with the most standard workflow.

Recall the architecture of a single-layer network as shown in :numref:`fig_single_neuron`.
The layer is said to be *fully-connected*
because each of its inputs is connected to each of its outputs
by means of a matrix-vector multiplication.


In Keras, the fully-connected layer is defined in the `Dense` class. Since we only want to generate a single scalar output, we set that number to 1.

It is worth noting that, for convenience,
Keras does not require us to specify
the input shape for each layer.
So here, we do not need to tell Keras
how many inputs go into this linear layer.
When we first try to pass data through our model,
e.g., when we execute `net(X)` later,
Keras will automatically infer the number of inputs to each layer.
We will describe how this works in more detail later.


In [24]:
# `keras` is the high-level API for TensorFlow
#net basically represents the entire network. Layers are added sequentially to it
net = tf.keras.Sequential()

"""
Dense implements the operation: output = activation(dot(input, kernel) + bias)
where activation is the element-wise activation function passed as the activation 
argument, kernel is a weights matrix created by the layer, and bias is a bias 
vector created by the layer (only applicable if use_bias is True).
"""
#basically (X*w)+b where X is the features
net.add(tf.keras.layers.Dense(1))

## Initializing Model Parameters

Before using `net`, we need to initialize the model parameters,
such as the weights and bias in the linear regression model.
Deep learning frameworks often have a predefined way to initialize the parameters.
Here we specify that each weight parameter
should be randomly sampled from a normal distribution
with mean 0 and standard deviation 0.01.
The bias parameter will be initialized to zero.


The `initializers` module in TensorFlow provides various methods for model parameter initialization. The easiest way to specify the initialization method in Keras is when creating the layer by specifying `kernel_initializer`. Here we recreate `net` again.


In [25]:
initializer = tf.initializers.RandomNormal(stddev=0.01)
net = tf.keras.Sequential()
#kernel_initializer initializes the weight matrix and is passed to Dense
net.add(tf.keras.layers.Dense(1, kernel_initializer=initializer))

The code above may look straightforward but you should note
that something strange is happening here.
We are initializing parameters for a network
even though Keras does not yet know
how many dimensions the input will have!
It might be 2 as in our example or it might be 2000.
Keras lets us get away with this because behind the scenes,
the initialization is actually *deferred*.
The real initialization will take place only
when we for the first time attempt to pass data through the network.
Just be careful to remember that since the parameters
have not been initialized yet,
we cannot access or manipulate them.


## Defining the Loss Function


The `MeanSquaredError` class computes the mean squared error, also known as squared $L_2$ norm.
By default it returns the average loss over examples.


In [26]:
loss = tf.keras.losses.MeanSquaredError()

## Defining the Optimization Algorithm


Minibatch stochastic gradient descent is a standard tool
for optimizing neural networks
and thus Keras supports it alongside a number of
variations on this algorithm in the `optimizers` module.
Minibatch stochastic gradient descent just requires that
we set the value `learning_rate`, which is set to 0.03 here.


In [27]:
#This takes the current weights and bias vectors (of randomly selected rows of data) and 
#then calculates w = w - lr*dw/batch_size
#similar for bias
trainer = tf.keras.optimizers.SGD(learning_rate=0.03)

## Training

You might have noticed that expressing our model through
high-level APIs of a deep learning framework
requires comparatively few lines of code.
We did not have to individually allocate parameters,
define our loss function, or implement minibatch stochastic gradient descent.
Once we start working with much more complex models,
advantages of high-level APIs will grow considerably.
However, once we have all the basic pieces in place,
the training loop itself is strikingly similar
to what we did when implementing everything from scratch.

To refresh your memory: for some number of epochs,
we will make a complete pass over the dataset (`train_data`),
iteratively grabbing one minibatch of inputs
and the corresponding ground-truth labels.
For each minibatch, we go through the following ritual:

* Generate predictions by calling `net(X)` and calculate the loss `l` (the forward propagation).
* Calculate gradients by running the backpropagation.
* Update the model parameters by invoking our optimizer.

For good measure, we compute the loss after each epoch and print it to monitor progress.


In [28]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        with tf.GradientTape() as tape:
            #needed to calculate gradient
            l = loss(net(X, training=True), y)
        grads = tape.gradient(l, net.trainable_variables)
        trainer.apply_gradients(zip(grads, net.trainable_variables))
    #Average loss calculated over all the features data
    #only to monitor overall progress
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000178
epoch 2, loss 0.000103
epoch 3, loss 0.000101


Below, we compare the model parameters learned by training on finite data
and the actual parameters that generated our dataset.
To access parameters,
we first access the layer that we need from `net`
and then access that layer's weights and bias.
As in our from-scratch implementation,
note that our estimated parameters are
close to their ground-truth counterparts.


In [29]:
w = net.get_weights()[0]
print('error in estimating w', true_w - tf.reshape(w, true_w.shape))
b = net.get_weights()[1]
print('error in estimating b', true_b - b)

error in estimating w tf.Tensor([-0.00080657 -0.00029039], shape=(2,), dtype=float32)
error in estimating b [6.0081482e-05]


## Summary


* Using TensorFlow's high-level APIs, we can implement models much more concisely.
* In TensorFlow, the `data` module provides tools for data processing, the `keras` module defines a large number of neural network layers and common loss functions.
* TensorFlow's module `initializers` provides various methods for model parameter initialization.
* Dimensionality and storage are automatically inferred (but be careful not to attempt to access parameters before they have been initialized).


## Exercises


1. Review the TensorFlow documentation to see what loss functions and initialization methods are provided. Replace the loss by Huber's loss.

[Discussions](https://discuss.d2l.ai/t/204)
